<a href="https://colab.research.google.com/github/DaianneS/projeto-ghibli/blob/main/Lista_Filmes_Ghibli_Assistidos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Lista de Filmes Assistidos do Studio Ghibli

link para app feito no streamlit:
https://projeto-ghibli-urnctdbat5ytzxr7sybhsm.streamlit.app/

Este notebook permite que você selecione os filmes do Studio Ghibli que você **já assistiu**, e gera:

-  Uma lista encadeada dos filmes selecionados.
-  Uma visualização gráfica da lista encadeada.
-  Uma tabela com informações dos filmes assistidos.

---


### Instalar bibliotecas

In [1]:
!pip install graphviz ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.7 MB/s eta 0:00:00


###Importar bibliotecas

In [2]:
import pandas as pd
import requests
from graphviz import Digraph
from IPython.display import Image, display, Markdown, clear_output
import ipywidgets as widgets

###Coletando dados da API

In [3]:
url = "https://ghibliapi.vercel.app/films"
response = requests.get(url)

if response.status_code == 200:
    dados_filmes = response.json()
else:
    print("Erro ao acessar a API")

###Classes da lista encadeada

In [4]:
#Representacao de cada nó da lista encadeada
class No:
    def __init__(self, dados):
        self.dados = dados
        self.proximo = None

#Operações da lista encadeada
class Lista:
    def __init__(self):
        self.cabeca = None

    #Checagem para ver se o filme já está na lista
    def contem(self, filme):
        atual = self.cabeca
        while atual:
            if atual.dados['id'] == filme['id']:
                return True
            atual = atual.proximo
        return False

    #Inserir nó no final da lista
    def inserir_final(self, dados):
        if self.contem(dados):
            return False  # Impede duplicados

        novo_no = No(dados)
        if self.cabeca is None:
            self.cabeca = novo_no
        else:
            atual = self.cabeca
            while atual.proximo:
                atual = atual.proximo
            atual.proximo = novo_no
        return True

    #remover o último elemento
    def remover_ultimo(self):
      if self.cabeca is None:
        return None  # Lista vazia

      if self.cabeca.proximo is None:
        removido = self.cabeca
        self.cabeca = None
        return removido

      atual = self.cabeca
      while atual.proximo.proximo:
        atual = atual.proximo

      removido = atual.proximo
      atual.proximo = None
      return removido

    #Exibir dados dos nós em formato de lista
    def exibir(self):
        dados_lista = []
        atual = self.cabeca
        while atual is not None:
            dados_lista.append(atual.dados)
            atual = atual.proximo
        return dados_lista

    #Gera grafico da lista encadeada
    def gerar_grafico(self, nome_arquivo="lista_encadeada"):
        grafico = Digraph(comment='Lista de Filmes Assistidos')
        atual = self.cabeca
        contador = 0

        while atual:
            label = f"{atual.dados['title']}\n({atual.dados['release_date']})"
            grafico.node(str(contador), label=label)

            if atual.proximo:
                grafico.edge(str(contador), str(contador + 1))

            atual = atual.proximo
            contador += 1

        #Salva o gráfico como PNG
        caminho_arquivo = grafico.render(nome_arquivo, format="png", cleanup=True)
        return caminho_arquivo

    #Limpar a lista
    def limpar(self):
        self.cabeca = None

###Interface

In [14]:
#Mensagem de explicação
display(Markdown(
"""
# Lista de Filmes Assistidos do Studio Ghibli

 **Instruções:**
- Marque os filmes que você **já assistiu** utilizando as caixas abaixo.
- Depois, clique no botão **"Gerar Lista de Filmes Assistidos"** para ver:
  - A lista encadeada em formato de texto.
  - A visualização gráfica da lista.
  - Uma tabela com os dados dos filmes assistidos.
"""
))

#Lista de checkbox
checkboxes = []
for i, filme in enumerate(dados_filmes):
    checkboxes.append(
        widgets.Checkbox(
            value=False,
            description=f"{i + 1}. {filme['title']} ({filme['release_date']}) - Diretor: {filme['director']}",
            indent=False,
            layout=widgets.Layout(width='auto')
        )
    )

#Organiza os checkboxes em uma caixa (Vbox) com bosrda e Padding
box = widgets.VBox(checkboxes,
                   layout=widgets.Layout(border='1px solid #1b2e2d', padding='10px', width='90%'))
display(box)


# Inicializa a lista encadeada
lista = Lista()

# Cria os botões
botao_adicionar = widgets.Button(description="➕ Adicionar Filmes", button_style='success', layout=widgets.Layout(width='40%', height='50px'))
botao_remover_ultimo = widgets.Button(description="❌ Remover Último", button_style='warning', layout=widgets.Layout(width='40%', height='50px'))
botao_limpar = widgets.Button(description="🗑️ Limpar Lista", button_style='danger', layout=widgets.Layout(width='40%', height='50px'))
botao_grafico = widgets.Button(description="📈 Gerar Gráfico", button_style='primary', layout=widgets.Layout(width='20%', height='50px'))
botao_tabela = widgets.Button(description="📊 Gerar Tabela", button_style='primary', layout=widgets.Layout(width='20%', height='50px'))

# Cria a área de saída
output = widgets.Output()

# Organiza os botões em linhas
botoes_linha1 = widgets.HBox([botao_adicionar, botao_remover_ultimo])
botoes_linha2 = widgets.HBox([botao_limpar, botao_grafico, botao_tabela])

display(botoes_linha1, botoes_linha2, output)

# Adicionar filmes selecionados
def adicionar_filmes(b):
    with output:
        clear_output()
        selecionados = [i for i, cb in enumerate(checkboxes) if cb.value]

        if selecionados:
            for indice in selecionados:
                filme = dados_filmes[indice]
                lista.inserir_final(filme)
                print(f" Adicionado: {filme['title']} ({filme['release_date']})")
        else:
            print(" Nenhum filme selecionado para adicionar.")

# Remover o último filme
def remover_ultimo(b):
    with output:
        clear_output()
        removido = lista.remover_ultimo()
        if removido:
            print(f" Filme removido: {removido.dados['title']} ({removido.dados['release_date']})\n")

            # Mostrar a lista atualizada em formato textual
            dados_atual = lista.exibir()
            if dados_atual:
                print(" Lista atualizada:")
                for i, filme in enumerate(dados_atual, 1):
                    print(f"{i}. {filme['title']} ({filme['release_date']}) - Diretor: {filme['director']}")
            else:
                print(" A lista agora está vazia.")
        else:
            print(" Lista vazia. Nenhum filme para remover.")

# Limpar a lista inteira
def limpar_lista(b):
    with output:
        clear_output()

        lista.limpar()  # Limpa a lista encadeada
        print(" Lista limpa com sucesso!")

        # Desmarca todas as checkboxes
        for cb in checkboxes:
            cb.value = False

# Gerar gráfico da lista
def gerar_grafico(b):
    with output:
        clear_output()
        if lista.cabeca is not None:
            caminho_imagem = lista.gerar_grafico()
            display(Image(filename=caminho_imagem))
        else:
            print(" A lista está vazia. Adicione filmes para gerar o gráfico.")

# Gerar tabela da lista
def gerar_tabela(b):
    with output:
        clear_output()
        dados_lista = lista.exibir()

        if dados_lista:
            df = pd.DataFrame(dados_lista)
            df = df[['title', 'director', 'release_date', 'rt_score']]
            df.columns = ['Título', 'Diretor', 'Ano', 'Score']
            display(df)
        else:
            print(" A lista está vazia. Adicione filmes para gerar a tabela.")

botao_adicionar.on_click(adicionar_filmes)
botao_remover_ultimo.on_click(remover_ultimo)
botao_limpar.on_click(limpar_lista)
botao_grafico.on_click(gerar_grafico)
botao_tabela.on_click(gerar_tabela)


# Lista de Filmes Assistidos do Studio Ghibli

 **Instruções:**
- Marque os filmes que você **já assistiu** utilizando as caixas abaixo.
- Depois, clique no botão **"Gerar Lista de Filmes Assistidos"** para ver:
  - A lista encadeada em formato de texto.
  - A visualização gráfica da lista.
  - Uma tabela com os dados dos filmes assistidos.


Output()